In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/home/mglee/anaconda3/envs/save/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
naver_comments = pd.read_csv("../../data/KP_naver_comments_monthlyTop4000.csv")
daum_comments = pd.read_csv("../../data/KP_daum_comments_monthlyTop800.csv")

네이버 포메팅

In [3]:
naver_comments['c_time'] = naver_comments.c_time.apply(lambda x : datetime.strptime(x, '%Y-%m-%dT%H:%M:%S+0900'))

In [4]:
naver_comments['ymd'] = naver_comments['c_time'].apply(lambda x : datetime.date(x))

In [5]:
naver_input = naver_comments[['url', 'text', 'ymd']]

다음 포메팅

In [6]:
daum_comments['createdAt'] = daum_comments['createdAt'].apply(lambda x : datetime.strptime(x, '%Y-%m-%dT%H:%M:%S+0900'))

In [7]:
daum_comments['ymd'] = daum_comments['createdAt'].apply(lambda x : datetime.date(x))

In [8]:
daum_comments.columns

Index(['Unnamed: 0', 'id', 'userId', 'postId', 'forumId', 'parentId', 'type',
       'status', 'flags', 'rating', 'icon', 'content', 'createdAt',
       'updatedAt', 'childCount', 'likeCount', 'dislikeCount',
       'recommendCount', 'screenedByKeeper', 'user', 'org_url', 'ymd'],
      dtype='object')

In [9]:
daum_input = daum_comments[['org_url', 'content', 'ymd']]

In [10]:
daum_input.columns = ['url', 'text', 'ymd']

In [11]:
news_comments_input = pd.concat([naver_input, daum_input])

In [12]:
news_comments_input

,url,text,ymd
0,https://n.news.naver.com/mnews/article/comment...,네이버 페이보다 수수료 낮을까?,2017-01-19
1,https://n.news.naver.com/mnews/article/comment...,공부해야할 과목이 또 하나 늘었군.. - IT종사자 -,2017-01-19
2,https://n.news.naver.com/mnews/article/comment...,"네이버도 정기결제, 포인트 적립 다 똑같이 되고 먼저 됐는데요.",2017-01-25
3,https://n.news.naver.com/mnews/article/comment...,"네이버페이 좋아요.네이버페이체크카드로 교통카드 사용하면 포인트주고, 천원써도 포인...",2017-01-22
4,https://n.news.naver.com/mnews/article/comment...,네이버페이는 편할거라 생각해서 썼는데 포인트도 제한 교통적립 되긴되는건가?카카오페이...,2017-01-22
...,...,...,...
71387,https://v.daum.net/v/20221226050109898,네이버가 짱이지 ㅋㅋ\n\n올개 30만원 모임\n\n얌얌,2022-12-26
71388,https://v.daum.net/v/20221229172340151,손절한 주식은???!!!,2022-12-29
71389,https://v.daum.net/v/20221229172340151,배당금이나 주라,2022-12-29
71390,https://v.daum.net/v/20221229172340151,어제 새벽부터 카카오페이 안된다!아직도 안되는 이유가 뭐냐?,2022-12-29


In [13]:
news_comments_input.to_csv("../../data/news_comments_date.csv")

In [2]:
import pandas as pd

import sys
sys.path.append("/home/mglee/VSCODE/git_folder/KakaoPay_ESG/code")

import comments_sentiment as senti

2023-03-02 11:39:26.322220: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


device : cuda:0
using cached model. /home/mglee/VSCODE/git_folder/KakaoPay_ESG/code/news/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/KakaoPay_ESG/code/news/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/mglee/VSCODE/git_folder/KakaoPay_ESG/code/news/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [3]:
news_comments_input = pd.read_csv("../../data/news_comments_date.csv")

In [10]:
esg_comments = pd.read_csv("../../data/ESG_split/comments_by_esg.csv")

In [4]:
news_comments_input['senti'] = senti.main(news_comments_input['text'])

BERT dataset creating......
BERT Classifier Start......


100%|██████████| 3219/3219 [03:42<00:00, 14.47it/s]


Predicting......


In [11]:
esg_comments['senti'] = senti.main(esg_comments['text'])

BERT dataset creating......
BERT Classifier Start......


100%|██████████| 4/4 [00:01<00:00,  3.57it/s]

Predicting......


In [17]:
news_comments_input['senti'].mean()

0.05447718264322591

In [24]:
news_comments_input['group'] = 'normal'

In [29]:
news_comments_input[['senti','group']]

,senti,group
0,0,normal
1,0,normal
2,0,normal
3,0,normal
4,1,normal
...,...,...
411984,0,normal
411985,0,normal
411986,0,normal
411987,0,normal


In [30]:
esg_comments['group'] = 'ESG'

In [46]:
pd.concat([esg_comments[['senti','group']], news_comments_input[['senti','group']]]).to_csv("../../result/normal_esg_senti.csv")

: 

In [6]:
news_comments_senti_ts = news_comments_input.groupby('ymd').sum('senti')

In [7]:
news_comments_count_ts = news_comments_input.groupby('ymd').size()

In [82]:
pd.DataFrame(news_comments_senti_ts['senti'] / news_comments_count_ts).to_csv("../../data/time series/댓글 감성분석 날짜별 비율 ts0302.csv")

In [86]:
news_comments_count_ts.to_csv("../../data/time series/댓글 개수 날짜별 ts0302.csv ")